In [1]:
from collections import defaultdict
import itertools
import re
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, BertForSequenceClassification

from tda4atd.stats_count import *
from tda4atd.grab_weights import grab_attention_weights, text_preprocessing

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
!env | grep CUDA_VISIBLE

In [4]:
np.random.seed(42) # For reproducibility.

In [5]:
max_tokens_amount  = 512 # The number of tokens to which the tokenized text is truncated / padded.
stats_cap          = 500 # Max value that the feature can take. Is NOT applicable to Betty numbers.
    
layers_of_interest = [i for i in range(12)]  # Layers for which attention matrices and features on them are 
                                             # calculated. For calculating features on all layers, leave it be
                                             # [i for i in range(12)].
stats_name = "s_e_v_c_b0b1" # The set of topological features that will be count (see explanation below)

thresholds_array = [0.025, 0.05, 0.1, 0.25, 0.5, 0.75] # The set of thresholds
thrs = len(thresholds_array)                           # ("t" in the paper)

model_path = "bert-base-uncased"  
tokenizer_path = "bert-base-uncased"  

# You can use either standard or fine-tuned BERT. If you want to use fine-tuned BERT to your current task, save the
# model and the tokenizer with the commands tokenizer.save_pretrained(output_dir); 
# bert_classifier.save_pretrained(output_dir) into the same directory and insert the path to it here.

In [6]:
from datasets import load_dataset

imdb = load_dataset("imdb")
x = 0
sentences = []
for elem in imdb["train"]:
    sentences.append(elem["text"])
sentences = np.array(sentences)
print(sentences.shape)
# print("Average amount of words in example:", \
#       np.mean(list(map(len, map(lambda x: re.sub('\w', ' ', x).split(" "), sentences)))))
# print("Max. amount of words in example:", \
#       np.max(list(map(len, map(lambda x: re.sub('\w', ' ', x).split(" "), sentences)))))
# print("Min. amount of words in example:", \
#       np.min(list(map(len, map(lambda x: re.sub('\w', ' ', x).split(" "), sentences)))))

Found cached dataset imdb (/home/sha43/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

(25000,)


In [7]:
data = pd.DataFrame({"sentence": sentences[:2048]})
sentences = data['sentence']


In [8]:
def get_token_length(batch_texts):
    inputs = tokenizer.batch_encode_plus(batch_texts,
       return_tensors='pt',
       add_special_tokens=True,
       max_length=MAX_LEN,             # Max length to truncate/pad
       pad_to_max_length=True,         # Pad sentence to max length
       truncation=True
    )
    inputs = inputs['input_ids'].numpy()
    n_tokens = []
    print("Counting lens")
    for i in tqdm(range(inputs.shape[0])):
        ids = np.argwhere(inputs[i] == tokenizer.pad_token_id)
        if not len(ids):
            n_tokens.append(MAX_LEN)
        else:
            n_tokens.append(ids[0, 0])
    return n_tokens

In [9]:
MAX_LEN = max_tokens_amount
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)

In [10]:
data['tokenizer_length'] = get_token_length(data['sentence'].values)
ntokens_array = data['tokenizer_length'].values

Counting lens


  0%|          | 0/2048 [00:00<?, ?it/s]

In [11]:
from math import ceil

batch_size = 2 # batch size
number_of_batches = ceil(len(data['sentence']) / batch_size)
DUMP_SIZE = 64 # number of batches to be dumped
batched_sentences = np.array_split(data['sentence'].values, number_of_batches)
adj_matricies = []
adj_filenames = []
assert number_of_batches == len(batched_sentences) # sanity check


In [12]:
device='cuda'
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)
MAX_LEN = max_tokens_amount

In [13]:
# What is calculated in "f(v)". You can add any other function from the array with vertex degrees.

def function_for_v(list_of_v_degrees_of_graph):
    return sum(map(lambda x: np.sqrt(x*x), list_of_v_degrees_of_graph))

def split_matricies_and_lengths(adj_matricies, ntokens_array, num_of_workers):
    splitted_adj_matricies = np.array_split(adj_matricies, num_of_workers)
    splitted_ntokens = np.array_split(ntokens_array, num_of_workers)
    print([len(elem) for elem in splitted_adj_matricies])
    print([len(elem) for elem in splitted_ntokens])
    assert all([len(m)==len(n) for m, n in zip(splitted_adj_matricies, splitted_ntokens)]), "Split is not valid!"
    return zip(splitted_adj_matricies, splitted_ntokens)

In [14]:
import os
from multiprocessing import Pool
from tqdm import tqdm

num_of_workers = 32
pool = Pool(num_of_workers)

Process ForkPoolWorker-8:
Process ForkPoolWorker-23:
Process ForkPoolWorker-13:
Process ForkPoolWorker-12:
Process ForkPoolWorker-11:
Process ForkPoolWorker-4:
Process ForkPoolWorker-18:
Process ForkPoolWorker-26:
Process ForkPoolWorker-6:
Process ForkPoolWorker-16:
Process ForkPoolWorker-5:
Process ForkPoolWorker-19:
Process ForkPoolWorker-10:
Process ForkPoolWorker-32:
Process ForkPoolWorker-2:
Process ForkPoolWorker-14:
Process ForkPoolWorker-20:
Process ForkPoolWorker-9:
Process ForkPoolWorker-15:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-17:
Process ForkPoolWorker-28:
Process ForkPoolWorker-24:
Process ForkPoolWorker-7:
Process ForkPoolWorker-29:
Process ForkPoolWorker-22:
Process ForkPoolWorker-21:
Process ForkPoolWorker-25:
Process ForkPoolWorker-31:
Process ForkPoolWorker-27:
Process ForkPoolWorker-30:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  Fil

  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sha43/projects/vkr/tda4atd/stats_count.py", line 210, in b0_b1
    g = nx.Graph(g_list[j].to_undirected())
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/process.

  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
KeyboardInterrupt
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File

  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/queues.py", line 365, in get
    res = self._reader.recv_bytes()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/sha43/anaconda3/envs/diploma/lib/python3.9/multiprocessing/synchronize.py", lin

In [15]:
models = ["bert-base-uncased"] + [f"./finetuned-scrumbled-wikitext2-3e-4/checkpoint-{i}" for i in range(80, 40000, 80)]
models = [f"./finetuned-scrumbled-wikitext2-3e-4/checkpoint-{i}" for i in range(4720 + 80, 40000, 80)]

for model_name in models:
    model = BertForSequenceClassification.from_pretrained(model_name, output_attentions=True)
    model = model.to(device)
    
    stats_tuple_lists_array = []
    adj_matricies = []
    for i in tqdm(range(number_of_batches), desc="Weights calc"): 
        attention_w = grab_attention_weights(model, tokenizer, batched_sentences[i], max_tokens_amount, device)
        # sample X layer X head X n_token X n_token
        adj_matricies.append(attention_w)
        if (i+1) % DUMP_SIZE == 0: # dumping
            adj_matricies = np.concatenate(adj_matricies, axis=1)
            adj_matricies = np.swapaxes(adj_matricies, axis1=0, axis2=1) # sample X layer X head X n_token X n_token
            batch_n = (i - 1) // DUMP_SIZE
            ntokens = ntokens_array[batch_n*batch_size*DUMP_SIZE : (batch_n+1)*batch_size*DUMP_SIZE]
            splitted = split_matricies_and_lengths(adj_matricies, ntokens, num_of_workers)
            args = [(m, thresholds_array, ntokens, stats_name.split("_"), stats_cap) for m, ntokens in splitted]
            stats_tuple_lists_array_part = pool.starmap(
                count_top_stats, args
            )
            stats_tuple_lists_array.append(np.concatenate([_ for _ in stats_tuple_lists_array_part], axis=3))
            adj_matricies = []
    print(f"Ignoring {len(adj_matricies)} sentences")
    stats_tuple_lists_array = np.concatenate(stats_tuple_lists_array, axis=3)
    with open(f"attention_features/{model_name.replace('./', '').replace('/', '_')}", "wb") as fout:
        np.save(fout, stats_tuple_lists_array)

Some weights of the model checkpoint at ./finetuned-scrumbled-wikitext2-3e-4/checkpoint-4800 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


Weights calc:  12%|█▏        | 127/1024 [05:18<05:17,  2.83it/s]  

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


Weights calc:  19%|█▊        | 191/1024 [10:04<02:43,  5.09it/s]   

...

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


Weights calc:  94%|█████████▎| 959/1024 [1:09:00<00:08,  7.62it/s]  

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


Weights calc:  94%|█████████▎| 959/1024 [1:13:05<04:57,  4.57s/it]


KeyboardInterrupt: 